In [1]:
#https://motius.breezy.hr/p/0b622aa1163e01-join-our-tech-community-as-working-student-m-f-d/apply?utm_source=pt-linkedin&utm_medium=job-posting

In [2]:
from transformers import  AutoModel,AutoTokenizer,AutoModelForMaskedLM , Trainer,TrainingArguments,\
BitsAndBytesConfig,pipeline,default_data_collator,DataCollatorWithPadding,DataCollatorForLanguageModeling
from transformers.utils import move_cache
from chat_template_utils import get_json_schema
from llama_cpp import Llama
from utils import *
from peft import *
import datasets
from datasets import Features, Value
import torchmetrics
import torch
from torch.utils.data.dataloader import DataLoader,Dataset
from hqq.engine.hf import HQQModelForCausalLM
from hqq.models.hf.base import AutoHQQHFModel
from huggingface_hub import snapshot_download
#import deepspeed
import os
import pandas as pd
import json
from var_dump import var_dump
from bs4 import BeautifulSoup,PageElement,Comment
import matplotlib.pyplot as plt
import seaborn as sns

cache_dir='/var/tmp/.cache/' #'/proj/ciptmp/ix05ogym/.cache/'
output_dir = cache_dir+'outputs/'

move_cache(cache_dir)


No ROCm runtime is found, using ROCM_HOME='/usr'
/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


0it [00:00, ?it/s]

In [3]:
df = pd.read_excel('alldata_cleaned.xlsx',dtype={'query':str,'element':str})
c = df['element'].str.contains('<form')
df = df[c].drop_duplicates(['query']).reset_index(drop=True).drop(columns=['Unnamed: 0'])
df

,query,element
0,Simply submit your CV and our system will auto...,<form>Personal informationCV or resumeBrowse C...
1,Thank you for your interest in mediaire Please...,"<form accept-charset=""UTF-8"" action=""https://c..."
2,Interested EmailFirst NameLast NameDocumentsdo...,"<form data-testid=""ApplyStep1Form"" method=""POS..."
3,Well send a link to this job to your email App...,"<form novalidate=""""><input autocomplete=""email..."
4,Apply No Sign Up CV Drop,"<form method=""post"" name=""New Form""><input id=..."
...,...,...
407,Search by Keyword\n\n\nSearch by Location\n\n\...,"<form action=""/search/"" lang=""en-GB"" method=""g..."
408,Search by Keyword\n\n\n\n\nSearch by Location\...,"<form action=""/search/"" lang=""en-US"" method=""g..."
409,Search\nSearch\n\n\nSubmit,"<form action=""https://www.intersystems.com/sea..."
410,Application Details\n\nWant to apply faster\nl...,"<form action=""https://usa.applybe.com/Job/Appl..."


In [4]:
df['query'] = df['query'].apply(lambda x: str(x))
df['element'] = df['element'].apply(lambda x: str(x))

df['query'] = df['query'].apply(lambda x: x.replace('\xa0', ' '))
df['element'] = df['element'].apply(lambda x: x.replace('\xa0', ' '))

In [5]:
from sentence_transformers  import SentenceTransformer
#from transformers import AutoModel
model_name= 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
#sentence-transformers/all-MiniLM-L12-v2
embdeding_tokenizer = AutoTokenizer.from_pretrained(model_name,cache_dir=cache_dir)

#embeding_model= AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L12-v2',cache_dir=cache_dir)#.to('cuda')
embeding_model= SentenceTransformer(model_name,cache_folder=cache_dir)#.to('cuda')
#guide_model= SentenceTransformer(model_name,cache_folder=cache_dir)#.to('cuda')



embeding_model.eval()
embeding_model

/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [6]:
def print_df(df):
    for i,x in df.iterrows():
        print(i)
        for k,v in x.items():
            if k=='query':
                continue
            print(k,v)
            
        print('---------------------')
    
c = df['query'].str.lower().str.contains('search')==True
df.loc[c,'label']='search form'
c = df['query'].str.lower().str.contains('submit application')==True
df.loc[c,'label']='application form'
c = df['query'].str.lower().str.contains('login')==True
df.loc[c,'label']='login form'
c = df['query'].str.lower().str.contains('register')==True
df.loc[c,'label']='register form'

labeled = df.dropna().reset_index(drop=True)
print(labeled['label'].value_counts())
print_df(labeled)

label
search form         81
register form       27
login form          14
application form     8
Name: count, dtype: int64
0
element <form method="post" name="applynon-Member">								First Name							<input aria-required="true" id="form-field-name" name="form_fields[name]" placeholder="First Name" required="required" size="1" type="text"/>								Last Name							<input aria-required="true" id="form-field-field_02fc995" name="form_fields[field_02fc995]" placeholder="Last Name" required="required" size="1" type="text"/>								City							<input aria-required="true" id="form-field-field_e992216" name="form_fields[field_e992216]" placeholder="Enter Location" required="required" size="1" type="text"/>								Email							<input aria-required="true" id="form-field-email" name="form_fields[email]" required="required" size="1" type="email"/>								CV							<input aria-required="true" data-maxsize="2" data-maxsize-message="This file exceeds the maximum allowed size." id="form-field-message" n

In [7]:
labels = labeled['label'].unique()
index2label = dict(enumerate(labels))
label2index= dict(zip(index2label.values(),index2label.keys()))

label2index

{'application form': 0, 'search form': 1, 'login form': 2, 'register form': 3}

In [8]:
labeled['label'] = labeled['label'].map(lambda x:label2index[x])


In [9]:
from sentence_transformers.trainer import SentenceTransformerTrainer,SentenceTransformerTrainingArguments
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


In [10]:
"""#data.loc[:,'label']=1
features = Features({
    'query': Value('string'),
    'element': Value('string'),
    'answer': Value('string'),
    'label': Value('int64')
})
train_dataset = datasets.Dataset.from_pandas(data,features)

train_dataset = train_dataset.remove_columns(['answer'])#,'label'])
"""

"""
train_dataset = train_dataset.map(lambda x:
    {
        'element':embdeding_tokenizer(x['element'],return_tensors='pt',truncation=True,max_length=512),
        'label':x['label']
        
    }
    ,
    remove_columns=['query']

    
    )
    
"""

"\ntrain_dataset = train_dataset.map(lambda x:\n    {\n        'element':embdeding_tokenizer(x['element'],return_tensors='pt',truncation=True,max_length=512),\n        'label':x['label']\n        \n    }\n    ,\n    remove_columns=['query']\n\n    \n    )\n    \n"

In [11]:
#labeled.loc[4,'query']="ss"
#labeled.loc[4,'label']=0
features = Features({
    'query': Value('string'),
    'element': Value('string'),
    'label': Value('int64')
})
"""labeled.loc[0:10,'query']="sss"
labeled.loc[8,'query']="skmklmss"""
"""
labeled.loc[0:10,'label']=0
labeled.loc[2,'label']=1
"""

train_dataset = datasets.Dataset.from_pandas(labeled[:128],features)
train_dataset = train_dataset.remove_columns(['element'])
train_dataset

Dataset({
    features: ['query', 'label'],
    num_rows: 128
})

In [12]:
train_dataset[0]

{'query': 'First Name\n\n\nLast Name\n\n\nCity\n\n\nEmail\n\n\nCV\n\n\nAdditional Files\n\n\n\nTelephone Number MobileWork\n\n\nDo You Require Visa Sponsorship\n Yes No \n\n\n \nSubmit Application',
 'label': 0}

In [13]:
"""def pd_full_screen():
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    
def pd_normal_screen():
    pd.set_option('display.max_rows', 10)
    pd.set_option('display.max_columns', 5)
pd_full_screen()
"""
train_dataset[0]

{'query': 'First Name\n\n\nLast Name\n\n\nCity\n\n\nEmail\n\n\nCV\n\n\nAdditional Files\n\n\n\nTelephone Number MobileWork\n\n\nDo You Require Visa Sponsorship\n Yes No \n\n\n \nSubmit Application',
 'label': 0}

In [17]:
from sentence_transformers.losses import BatchSemiHardTripletLoss as myloss
trainer  =  SentenceTransformerTrainer(model=embeding_model,
                           train_dataset=train_dataset,
                           tokenizer=embdeding_tokenizer,
                           loss= myloss(embeding_model)#,384,len(u)),#,'sentence-transformers/all-MiniLM-L12-v2',True),
                           ,args=SentenceTransformerTrainingArguments(
                               output_dir=output_dir,
                               num_train_epochs=20,
                               per_device_train_batch_size=32,
                               logging_steps=1,
                               #learning_rate=1e-9
                               
                           )
                           
                           )


trainer.train()

Step,Training Loss
1,0.247800
2,0.022100
3,0.054700
4,1.086900
5,0.030900
6,0.424700
7,0.214500
8,0.012900
9,0.168300
10,0.187900


TrainOutput(global_step=80, training_loss=0.05501729866718961, metrics={'train_runtime': 9.5477, 'train_samples_per_second': 268.128, 'train_steps_per_second': 8.379, 'total_flos': 0.0, 'train_loss': 0.05501729866718961, 'epoch': 20.0})

In [15]:
#c = test_dataset['query'].isna()
#test_dataset.loc[c,'query']='cv'
#test_dataset.to_excel('forms.xlsx',index=0)

In [16]:
test_dataset = pd.read_excel('forms.xlsx')
c = test_dataset['label'].notna()
test_dataset = test_dataset.loc[c,:].reset_index(drop=True)

tp = test_dataset #train_dataset.to_pandas()
def compute_metric():
    embeding_model.eval()
    acc=0
    
    with torch.no_grad():
        
        l=embeding_model.encode(LABEL_INDEX_TO_KEY)
        l = torch.tensor(l)
        print(l.shape)
        
        for i,x in tp.iterrows():
            #print(x)
            #for i,x in train_dataset.iterrows():
            #te = embdeding_tokenizer([x['element']],max_length=512,truncation=True,return_tensors='pt')['input_ids']
            #print(i)
            #x['label']
            o = embeding_model.encode( [x['query']] )
            o2 = embeding_model.encode( x['element'] )
            
            o = torch.tensor(o)
            o2 = torch.tensor(o2)
            
            sim = torch.cosine_similarity(l,o)
            #print(o2.shape)
            #sim = torch.cosine_similarity(o2,o,0)

            m = sim.argmax()
            tp.loc[i,'sudo_label']=LABEL_INDEX_TO_KEY[m]
            #tp.loc[i,'sudo_label_value']=sim
            if LABEL_INDEX_TO_KEY[m]==x['label']:
                acc+=1
                pass
            #print(LABEL_INDEX_TO_KEY[sim.argmax()],x['label'])
            
           
             
    
    acc/=len(test_dataset)
    print("acc:",acc)
    
    
    
compute_metric()

FileNotFoundError: [Errno 2] No such file or directory: 'forms.xlsx'

In [ ]:
test_dataset.loc[11,'query']='cv *'
tp

In [ ]:
for i,x in tp.iterrows():
            print(x['query'])
            print(x['element'])
            print(x['sudo_label'])
            #print(x['sudo_label_value'])
            print('-----------------------------------')